# CpNa with GPAW

This is an xHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up. Specifically it is a demonstration of skipping atoms, which are disordered on a special position can be skipped from the calculation with skip_symm in the computation_dict. In this case we are fortunate enough, that this is all it takes to lead the refinement to a reasonable result. Otherwise restraints would be needed, which is something, that is not implemented at the moment

In [1]:
from xharpy import (
    cif2data, xd_hkl2pd, lst2constraint_dict, create_construction_instructions,
    refine, write_cif, write_fcf, write_res
)
import os

In [2]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
filename = 'xharpy'

In [3]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [4]:
hkl = xd_hkl2pd('xd.hkl')

In [5]:
constraint_dict = lst2constraint_dict(
    'iam.lst'
)
constraint_dict['H1'] = constraint_dict['C1']

In [6]:
refinement_dict = {
    'f0j_source': 'gpaw_mpi',
    'reload_step': 1,
    'core': 'constant',
}

In [7]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [8]:
computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.15,
    'gridinterpolation': 4,
    'average_symmequiv': True,
    'convergence':{'density': 1e-7},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2,
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw'),
    'skip_symm': {
        2: [2, 3, 6, 7],
        3: [2, 3, 6, 7],
        4: [2, 3, 6, 7]
    }
}
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-06 15:43:39.164178
Preparing
  calculating the core structure factor by spline for Na
  calculating the core structure factor by spline for N
  calculating the core structure factor by spline for H
  calculating the core structure factor by spline for C
  calculating first atomic form factors
  calculated density with energy -511.40281675315276
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.123534, number of iterations: 2
  minimizing least squares sum
  wR2: 0.038814, number of iterations: 220
step 1: calculating new structure factors
  calculated density with energy -545.1062902191113
  minimizing least squares sum
  wR2: 0.032020, number of iterations: 139
step 2: calculating new structure factors
  calculated density with energy -544.7884561771557
  minimizing least squares sum
  wR2: 0.032189, number of iterations: 132
step 3: calculating new structure factors
  calculated density with energy -544.8551437

KeyboardInterrupt: 

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_res(
    out_res_path=os.path.join(output_folder, filename + '_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [ ]:
write_cif(
    output_cif_path=os.path.join(output_folder, filename + '.cif'),
    cif_dataset=filename,
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)